# Part 06b: Visualize seattle street groups by type
michael babb  
2024 11 24

In [1]:
# standard
import os

In [2]:
# external
import geopandas as gpd
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# custom
import run_constants as rc
from utils import *

# load the working seattle street network data

In [4]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [5]:
gdf = gpd.read_file(filename = fpn)

In [6]:
gdf.shape

(25773, 39)

In [7]:
gdf['ord_stname_concat'].unique().shape

(2455,)

In [8]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion',
       'ord_stname_unique', 'geometry'],
      dtype='object')

In [9]:
gdf['city_portion'].unique()

array(['CNTR', 'E', 'NE', 'NW', 'S', 'SW', 'W', 'N'], dtype=object)

## load the manually created street group data
These data were created in Step 6a, but we'll bring them into the Step 3 workflow.

In [10]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S03_STREET_GROUP_IN_FILE_NAME)

In [11]:
index_cols = ['sort_order', 'ord_street_name', 'ord_street_type', 'city_portion']
sg_df = pd.read_excel(io = fpn, index_col=[0, 1, 2, 3])
sg_df = sg_df.reset_index()

In [12]:
drop_cols = ['sort_order', 'progress', 'count']
sg_df = sg_df.drop(labels = drop_cols, axis = 1)


In [13]:
test_join = pd.merge(left = gdf, right = sg_df, how = 'left', indicator=True)
test_join['_merge'].value_counts()

_merge
both          19032
left_only      6741
right_only        0
Name: count, dtype: int64

In [14]:
gdf = test_join.copy()

In [15]:
gdf['group_id'] = gdf['group_id'].fillna(1).astype(int)
gdf.shape

(25773, 41)

# create an id across street groups

In [16]:
gdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,sndseg_update,compkey,comptype,unitid,unitid2,city_portion,ord_stname_unique,geometry,group_id,_merge
0,12378,12359,21329,1,1,1,0,0,1,1,...,2007-04-18 00:00:00+00:00,1284,68,00145,0015,CNTR,10TH AVE CNTR,"LINESTRING (-122.31942 47.6027, -122.31942 47....",2,both
1,12319,12265,21259,1,1,1,0,0,1,1,...,2007-04-20 00:00:00+00:00,1286,68,00145,0020,CNTR,10TH AVE CNTR,"LINESTRING (-122.31942 47.60352, -122.31943 47...",2,both
2,11809,11752,20601,1,1,1,0,0,1,1,...,2007-04-20 00:00:00+00:00,1194,68,00145,0112,CNTR,10TH AVE CNTR,"LINESTRING (-122.31938 47.61218, -122.31939 47...",2,both
3,12265,12222,21189,1,1,1,0,0,1,1,...,2007-04-20 00:00:00+00:00,1191,68,00145,0030,CNTR,10TH AVE CNTR,"LINESTRING (-122.31943 47.60437, -122.31943 47...",2,both
4,12222,12162,21131,1,1,1,0,0,1,1,...,2007-04-20 00:00:00+00:00,1192,68,00145,0040,CNTR,10TH AVE CNTR,"LINESTRING (-122.31943 47.60521, -122.31943 47...",2,both


In [17]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion',
       'ord_stname_unique', 'geometry', 'group_id', '_merge'],
      dtype='object')

In [18]:
col_names = ['ord_street_name', 'ord_street_type', 'ord_stname_concat',
             'ord_stname_unique', 'city_portion', 'group_id']
id_df = gdf[col_names].drop_duplicates()

In [19]:
id_df['ord_stname_type'] = id_df['ord_street_name'] + ' ' + id_df['ord_street_type']

In [20]:
id_df['sort_order'] = id_df['ord_street_name'].map(get_sort_order)

In [21]:
id_df = id_df.sort_values(by = ['sort_order', 'ord_street_type', 'city_portion'])

In [22]:
id_df.head()

,ord_street_name,ord_street_type,ord_stname_concat,ord_stname_unique,city_portion,group_id,ord_stname_type,sort_order
1647,1ST,AVE,1ST AVE,1ST AVE CNTR,CNTR,2,1ST AVE,001
1672,1ST,AVE,1ST AVE N,1ST AVE N,N,2,1ST AVE,001
1709,1ST,AVE,1ST AVE NE,1ST AVE NE,NE,2,1ST AVE,001
1764,1ST,AVE,1ST AVE NW,1ST AVE NW,NW,1,1ST AVE,001
1838,1ST,AVE,1ST AVE S,1ST AVE S,S,2,1ST AVE,001


In [23]:
id_df['ord_stname_unique'].unique().shape

(2457,)

In [24]:
# let's get the groups! can we aggregate this?

In [25]:
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'ord_stname_type', 'group_id', 'city_portion']
test_agg = id_df[col_names].groupby(col_names[:-1]).agg(city_portion_group = ('city_portion', lambda x: '_'.join(sorted(set(x)))),
                                                        n_groups = ('city_portion', lambda x: len(set(x)))).reset_index()

In [26]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [27]:
test_agg['city_portion_group'].unique().shape

(32,)

In [28]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [29]:
test_cpg = 'CNTR_W'
test_ost = 'BR'
check_df = test_agg.loc[(test_agg['city_portion_group'] == test_cpg) &
                        (test_agg['ord_street_type'] == test_ost), ]
print(check_df.shape)
check_df.head()

(0, 7)


,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups


In [30]:
# export to excel
city_portion_check = test_agg[['city_portion_group', 'ord_street_type', 'n_groups']].drop_duplicates().reset_index(drop = True)

In [31]:
city_portion_check.head()

,city_portion_group,ord_street_type,n_groups
0,NW_SW_W,AVE,3
1,CNTR_N_NE_S,AVE,4
2,NE,PL,1
3,SW,LN,1
4,SW,PL,1


In [32]:
city_portion_check.shape

(126, 3)

In [33]:
city_portion_check = city_portion_check.sort_values(by = ['city_portion_group', 'ord_street_type'])

In [34]:
city_portion_check.to_excel(excel_writer='city_portion_check.xlsx', index = False)

In [35]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_portion_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [36]:
test_agg.shape

(1816, 7)

In [37]:
# join back to the working gdf

In [38]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion',
       'ord_stname_unique', 'geometry', 'group_id', '_merge'],
      dtype='object')

In [39]:
gdf = gdf.drop(labels = ['_merge'], axis = 1)

In [40]:
test_join = pd.merge(left = gdf, right = test_agg, how = 'outer', indicator=True)

In [41]:
test_join['_merge'].value_counts()

_merge
both          25773
left_only         0
right_only        0
Name: count, dtype: int64

In [42]:
test_join.shape

(25773, 45)

In [43]:
test_join.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_portion',
       'ord_stname_unique', 'geometry', 'group_id', 'sort_order',
       'ord_stname_type', 'city_portion_group', 'n_groups', '_merge'],
      dtype='object')

In [44]:
test_join['_merge'].value_counts()

_merge
both          25773
left_only         0
right_only        0
Name: count, dtype: int64

In [45]:
gdf = test_join.copy()

# MAKE A PLOT OF THE DIFFERENT CITY GROUPS BY STREET TYPE

In [53]:
# use the same colors for each city portion
cdm = {}
dir_list = ['CNTR', 'E', 'N', 'NE', 'NW', 'S', 'SW', 'W']
for idir, dir_value in enumerate(dir_list):
    cdm[dir_value] = mpl.colormaps["Dark2"].colors[idir]

my_cmap = mpl.colors.ListedColormap([cdm[c] for c in dir_list])

In [54]:
wgdf = gdf.loc[gdf['n_groups'] > 1, :].copy()

In [ ]:
wgdf.shape

In [56]:
col_names = ['city_portion_group', 'ord_street_type']
control_df = wgdf[col_names].drop_duplicates()

In [ ]:
control_df.shape

In [ ]:
control_df.head()

In [59]:
# plot bounds to use across each plot
bounds = [-122.4197794277490061,47.4803548409661005, -122.2200188105690017,47.7341482423694004]

In [60]:
# load the boundaries of the city sections
ifpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTIONS_OUT_FILE_NAME)
cs_gdf = gpd.read_file(filename=ifpn)

In [ ]:
control_df.head()

In [62]:
ost = 'BR'
pgdf = gdf.loc[(gdf['ord_street_type'] == ost), :]

In [ ]:
pgdf['city_portion_group'].unique()

In [ ]:
pgdf.head()

In [ ]:
make_plots = True
if make_plots:
    for cr, crow in control_df.iterrows():
        cpg = crow['city_portion_group']
        ost = crow['ord_street_type']

        output_file_name = f"{ost}_{cpg}.png"
        ofpn = os.path.join(rc.S06_PLOT_OUTPUT_FILE_PATH_CITY_PORTION_GROUPS, output_file_name)
        print(ofpn)

        # now, let's make a map...
        pgdf = gdf.loc[(gdf['city_portion_group'] == cpg) &
                    (gdf['ord_street_type'] == ost), :]
        to_draw = pgdf[['city_portion', 'geometry']].dissolve(by = 'city_portion', as_index = False)
        #to_draw['coords'] = to_draw['geometry'].map(lambda x: x.centroid.coords[0])
        fig = plt.figure(layout = 'constrained', figsize = (5, 10))
        gs = GridSpec(1,1, figure = fig, height_ratios = [1])
        ax1 = fig.add_subplot(gs[0,0])
        ax1.set_xlim(bounds[0], bounds[2])
        ax1.set_ylim(bounds[1], bounds[3])
        cs_gdf.plot(ax = ax1, column = 'city_portion', alpha = .2)
        to_draw.plot(ax = ax1, column = 'city_portion', cmap = my_cmap,  linewidth = 5, legend = True)

        #for irrow, row in to_draw.iterrows():    
        #    ax1.annotate(text=row['city_portion'], xy=row['coords'], fontsize = 16 )

        ax1.set_axis_off()
        
        my_title = f"Street Type: {ost} | City Portion Group {cpg}"
        plt.title(label = my_title)

        #plt.show()

        fig.savefig(fname = ofpn)
        plt.close()